# Imports

To refresh imports

In [1]:
%reload_ext autoreload
%autoreload 2

Imports

In [2]:
# -------------------------------- torch stuff ------------------------------- #
import torch
from torch.utils.data import Dataset, DataLoader


# ----------------------------------- other ---------------------------------- #
from glob import glob
import soundfile as sf
from scipy.io import wavfile
import json
import os
from tqdm import tqdm
import sys

# ---------------------------------- Custom ---------------------------------- #
from utils.precision_loss import show_sum_error

### Why using scipy.io instead soundfile

In [3]:
print("soundfile converts directly to float 64")
sf.read("../data/LibriCount/10_85b5ac.wav")[0]

soundfile converts directly to float 64


array([ 0.0944519 ,  0.09963989,  0.13208008, ..., -0.05599976,
       -0.04922485, -0.03912354])

In [4]:
print("wavfile concervs the original format : int16")
wavfile.read("../data/LibriCount/10_85b5ac.wav")[1]

wavfile concervs the original format : int16


array([ 3095,  3265,  4328, ..., -1835, -1613, -1282], dtype=int16)

### Format

In [5]:
F16 = torch.float16
F32 = torch.float32
F64 = torch.float64

In [6]:
show_sum_error()

Conversion Errors from int16 in range(-32768, 32768) maxed by (max is=) :
	Float 16  8.0
	Float 32  0.0
	Float 64  0.0


# Consts

In [7]:
data_dir = "../data/LibriCount"
FTYPE = F32

# Dataset

In [8]:
class AudioCountGender(Dataset):
    def __init__(self, data_dir=data_dir, dtype = FTYPE, cache=True):
        self.sounds = glob(os.path.join(data_dir,"*.wav"))
        self.labels = glob(os.path.join(data_dir,"*.json"))
        self.dtype = dtype
        self.cache = cache
        if self.cache:
            self.data = []
            for index in range(len(self.sounds)):
                sample_rate, clip = wavfile.read(self.sounds[index])
                with open(self.labels[index]) as f:
                    label = json.load(f)
                genders = [0, 0] #[Male, Female]
                for person in label:
                    gender = person["sex"]
                    if gender == "F":
                        genders[1] += 1
                    else :
                        genders[0] += 1
                self.data.append([torch.tensor(clip, dtype=self.dtype).unsqueeze(0), torch.tensor(genders, dtype=self.dtype)]) #unsqueeze serves for channel = 1
                
                
    def __getitem__(self, index):
        if self.cache:
            return self.data[index]
        else:
            # clip, sample_rate = sf.read(self.sounds[index])
            sample_rate, clip = wavfile.read(self.sounds[index])
            with open(self.labels[index]) as f:
                label = json.load(f)
            genders = [0, 0] #[Male, Female]
            for person in label:
                gender = person["sex"]
                if gender == "F":
                    genders[1] += 1
                else :
                    genders[0] += 1
            return torch.tensor(clip, dtype=self.dtype).unsqueeze(0), torch.tensor(genders, dtype=self.dtype) #unsqueeze serves for channel = 1
    def __len__(self):
        return len(self.sounds)

In [9]:
data = AudioCountGender()
data[800]

[tensor([[ 3095.,  3265.,  4328.,  ..., -1835., -1613., -1282.]]),
 tensor([3., 7.])]

# Dataloader

In [10]:
dataloader = DataLoader(dataset=data, batch_size=8, shuffle=True, num_workers=8)

# Measures

In [11]:
from time import time
# L = []
start = time()
for d in dataloader:
    # L.append(d)
    print(len(d[0]), end="_")
end = time()
print()
print("Duration: {:.2f}s".format(end-start))
#Energy efficient mode
#Duration: 55.97s for F16 and no caching and list appending
#Duration: 1.72s for F16 and caching and list appending
#Equilibre mode 
#Duration: 12.46s for F16 and no caching and no list appending
#Duration: 0.88s for F16 and caching andn no list appending

8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_8_

# Simple model

In [12]:
# torch sequential
c_1 = 64
kernel_1 = 21
c_2 = 64
kernel_2 = 7
kernel_pool = 3

c_3 = 128
kernel_3 = 3
c_4 = 256
kernel_4 = 3
model = torch.nn.Sequential( #input size = 80000
    torch.nn.Conv1d(1, c_1, kernel_1, stride=5, dtype=FTYPE),
    torch.nn.Conv1d(c_1, c_2, kernel_2, dtype=FTYPE),
    torch.nn.ReLU(),
    torch.nn.Dropout(),
    torch.nn.MaxPool1d(kernel_pool),
    
    torch.nn.Conv1d(c_2, c_3, kernel_3, dtype=FTYPE),
    torch.nn.Conv1d(c_3, c_4, kernel_4, dtype=FTYPE),
    torch.nn.ReLU(),
    torch.nn.Dropout(),
    torch.nn.MaxPool1d(kernel_pool),
    
    torch.nn.Flatten(),
    torch.nn.Linear(c_4*1775, 2, dtype=FTYPE),
)

In [14]:
example = d[0].to("cuda")
model.to("cuda")

print("Shape : ", model.forward(example).shape)
model.forward(example)

Shape :  torch.Size([8, 2])


tensor([[  362.2938,  -189.9754],
        [  -26.9462,    -9.9904],
        [  -10.2068,     4.6790],
        [  -16.9262, -1272.8457],
        [ -737.4520,  -625.8914],
        [  805.6327,  -359.5741],
        [   91.7435,  -314.7892],
        [  381.6484,  -208.7020]], device='cuda:0', grad_fn=<AddmmBackward0>)

# Simple train

In [17]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 20
model.to("cuda")

for epoch in range(EPOCHS):
    mean_loss = 0
    for d in tqdm(dataloader):
        audios = d[0].to("cuda")
        labels = d[1].to("cuda")
        predictions = model.forward(audios)
        loss_value = loss(predictions, labels)
        mean_loss += loss_value.item()
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
    print("Epoch {}/{}".format(epoch+1, EPOCHS))
    print("Train Loss : {:.4f}".format(mean_loss/len(dataloader)))

100%|██████████| 715/715 [00:52<00:00, 13.70it/s]


Epoch 1/20
Train Loss : 4.4564


100%|██████████| 715/715 [00:51<00:00, 13.98it/s]


Epoch 2/20
Train Loss : 3.8403


100%|██████████| 715/715 [00:50<00:00, 14.03it/s]


...

Epoch 6/20
Train Loss : 3.5863


100%|██████████| 715/715 [00:50<00:00, 14.12it/s]


Epoch 7/20
Train Loss : 3.5863


 59%|█████▉    | 425/715 [00:30<00:20, 13.95it/s]


KeyboardInterrupt: 